<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=cce2872cc7e9c28466692bdb04f535d51ac1d51d5eca42a7522a94e170d893b3
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-11 15:08:50
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.18 L
Current:  1.41 C
-------------------
Today PnL: 20.70 K (0.15%)
Current PnL: -17.71 L (-11.92%)
CY Booked + Current PnL: -6.68 L (-4.5%)
-------------------
Total profit:  2.03 L
Total loss:  -19.74 L
-------------------
Total Booked + Current PnL: 20.30 L (16.68%)
Total Booked PnL: 38.01 L (31.23%)
Curr Year Booked PnL: 11.03 L (7.8%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 83.82 L (59.24%)
Deployed:  1.22 C
Current:  1.41 C
CAGR/XIRR %: 9.43%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,APOLLOHOSP,8285.00,-13.31,60.0,H-LC,11.20,173272.0,17095.0,8993.0,-0.67,10.95,5.19,16.71,22.0,1.90,1.26,30.29,X40N,BTT,HEALTHCARE
62,SBIN,863.00,-11.34,61.0,M-LC,4.29,218188.0,16708.0,10517.0,0.63,8.29,4.82,13.51,60.0,1.59,1.59,21.93,XY25,NTT,BANKS
77,TTKPRESTIG,770.00,105.76,62.0,M-SC,5.27,88608.0,-12169.0,12263.0,-0.03,-12.08,13.84,0.09,245.0,-0.99,0.65,16.41,OX40N,NTT,DURABLES
49,NATIONALUM,247.44,-42.83,71.0,H-MC,3.48,113658.0,13154.0,17481.0,2.54,13.09,15.38,30.49,79.0,0.75,0.83,49.73,MH,ATH,METALS
41,ITC,452.00,-37.08,56.0,H-LC,3.02,202813.0,2675.0,18213.0,0.41,1.34,8.98,10.44,4.0,0.15,1.48,6.97,X40,NTT,FMCG


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,43.54,64.0,M-MC,7.00,227149.0,2187.0,165478.0,1.69,0.97,72.85,74.53,192.0,0.01,1.65,33.54,XY24,BTT,STEEL
35,ICICIPRULI,790.00,-21.45,40.0,H-MC,1.99,135216.0,-572.0,43323.0,-0.66,-0.42,32.04,31.48,107.0,-0.01,0.98,11.65,X40,ATH,INSURANCE
51,PGHH,17973.08,-31.08,51.0,H-MC,4.62,199665.0,-1155.0,69923.0,0.42,-0.58,35.02,34.25,80.0,-0.02,1.45,4.59,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-15.98,39.0,H-LC,6.30,302731.0,-13111.0,125785.0,-0.27,-4.15,41.55,35.67,5.0,-0.10,2.20,9.13,X40N,ATH,FMCG
43,JIOFIN,387.00,-1.72,43.0,H-LC,13.62,267116.0,-249.0,65711.0,0.18,-0.09,24.60,24.48,48.0,-0.00,1.95,54.83,XY24,BTT,FINANCE
56,RELIANCE,1533.00,-13.66,48.0,H-LC,6.30,215748.0,5382.0,23409.0,0.44,2.56,10.85,13.68,37.0,0.23,1.57,19.50,XY25,NTT,REFINERIES
31,HCLTECH,1943.91,0.65,48.0,H-LC,10.16,230256.0,-11668.0,74948.0,0.18,-4.82,32.55,26.15,8.0,-0.16,1.68,8.81,X40,ATH,IT
80,UNITDSPR,1644.00,-13.52,49.0,H-LC,8.13,226782.0,-7362.0,55992.0,1.15,-3.14,24.69,20.77,86.0,-0.13,1.65,3.56,X40N,NTT,BREWERIES


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.20,-4.05,58.0,H-LC,1.26,237938.0,-8213.0,87418.0,-0.37,-3.34,36.74,32.18,16.0,-0.09,1.73,31.50,X200,ATH,IT
41,ITC,452.00,-37.08,56.0,H-LC,3.02,202813.0,2675.0,18213.0,0.41,1.34,8.98,10.44,4.0,0.15,1.48,6.97,X40,NTT,FMCG
50,NESTLEIND,1377.00,-7.08,63.0,H-LC,3.35,285878.0,20452.0,36335.0,0.35,7.71,12.71,21.40,11.0,0.56,2.08,16.09,XY25,NTT,FMCG
20,CIPLA,1795.00,-19.78,52.0,H-LC,5.95,213775.0,9275.0,32130.0,-0.22,4.54,15.03,20.25,10.0,0.29,1.56,13.66,X40N,BTT,PHARMA
83,VBL,671.64,-15.98,39.0,H-LC,6.30,302731.0,-13111.0,125785.0,-0.27,-4.15,41.55,35.67,5.0,-0.10,2.20,9.13,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
27,GREENPANEL,537.00,250.78,78.0,M-SC,6.14,156575.0,-26503.0,105485.0,17.61,-14.48,67.37,43.14,230.0,-0.25,1.14,47.28,XY24,NTT,MISC
57,REPCOHOME,880.00,-59.76,55.0,H-SC,7.91,181656.0,-54087.0,243383.0,4.11,-22.94,133.98,80.30,134.0,-0.22,1.32,22.97,XY24,NTT,FINANCE
32,HINDZINC,730.22,18.93,59.0,M-LC,1.77,193966.0,-11110.0,123673.0,3.01,-5.42,63.76,54.89,52.0,-0.09,1.41,17.37,X5K,ATH,METALS
14,BERGEPAINT,680.00,-17.10,54.0,H-MC,6.78,222730.0,-4609.0,52676.0,2.67,-2.03,23.65,21.14,106.0,-0.09,1.62,25.54,XY24,NTT,PAINTS
49,NATIONALUM,247.44,-42.83,71.0,H-MC,3.48,113658.0,13154.0,17481.0,2.54,13.09,15.38,30.49,79.0,0.75,0.83,49.73,MH,ATH,METALS


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
4,ANGELONE,3033.0,2.55,32.0,H-SC,7.10,181532.0,-9474.0,67167.0,-5.26,-4.96,37.00,30.21,157.0,-0.14,1.32,14.89,X40N,NTT,FINANCE
85,WHIRLPOOL,2270.0,-41.04,51.0,M-SC,1.98,99742.0,8244.0,70508.0,-2.91,9.01,70.69,86.07,223.0,0.12,0.73,44.84,XR,NTT,DURABLES
67,SONACOMS,1006.0,-38.28,44.0,M-SC,11.98,76886.0,-24273.0,99160.0,-2.38,-23.99,128.97,74.03,202.0,-0.24,0.56,8.42,AR,BTT,AUTO
82,VALIANTORG,838.0,-325.20,54.0,H-SC,8.41,125273.0,-38332.0,159648.0,-2.36,-23.43,127.44,74.15,139.0,-0.24,0.91,60.58,XR,NTT,CHEMICALS
81,VAIBHAVGBL,521.0,51.52,59.0,H-SC,10.24,131451.0,-51324.0,162395.0,-2.32,-28.08,123.54,60.77,125.0,-0.32,0.96,17.74,XR,NTT,JEWELLERY


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,101.87,64.0,H-SC,12.70,127818.0,-14661.0,34179.0,-2.09,-10.29,26.74,13.70,163.0,-0.43,0.93,50.86,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,105.76,62.0,M-SC,5.27,88608.0,-12169.0,12263.0,-0.03,-12.08,13.84,0.09,245.0,-0.99,0.65,16.41,OX40N,NTT,DURABLES
72,TATAELXSI,9161.0,-19.17,56.0,H-MC,8.05,109383.0,-22452.0,64678.0,-1.48,-17.03,59.13,32.03,98.0,-0.35,0.80,22.52,OX40N,NTT,IT
19,CERA,9475.0,-20.05,46.0,H-SC,2.23,145878.0,-30025.0,72049.0,-0.49,-17.07,49.39,23.89,149.0,-0.42,1.06,25.98,OX40N,NTT,CERAMICS
44,KANSAINER,340.0,-67.79,58.0,H-SC,2.27,222192.0,-47475.0,83811.0,1.06,-17.61,37.72,13.47,138.0,-0.57,1.62,12.38,XY24,NTT,PAINTS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4998.00,-26.44,54.0,H-LC,13.70,293628.0,-52328.0,176177.0,0.44,-15.13,60.00,35.80,1.0,-0.30,2.14,4.02,X40,BTT,IT
39,INFY,2275.00,-17.44,53.0,H-LC,9.76,321098.0,8210.0,163471.0,-1.64,2.62,50.91,54.87,3.0,0.05,2.34,9.45,X40,BTT,IT
41,ITC,452.00,-37.08,56.0,H-LC,3.02,202813.0,2675.0,18213.0,0.41,1.34,8.98,10.44,4.0,0.15,1.48,6.97,X40,NTT,FMCG
83,VBL,671.64,-15.98,39.0,H-LC,6.30,302731.0,-13111.0,125785.0,-0.27,-4.15,41.55,35.67,5.0,-0.10,2.20,9.13,X40N,ATH,FMCG
1,ABB,7934.00,-40.06,53.0,H-LC,12.28,250368.0,-11251.0,130467.0,1.05,-4.30,52.11,45.57,7.0,-0.09,1.82,6.53,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,43.54,64.0,M-MC,7.00,227149.0,2187.0,165478.0,1.69,0.97,72.85,74.53,192.0,0.01,1.65,33.54,XY24,BTT,STEEL
15,BLUESTARCO,2326.38,9.32,53.0,H-MC,3.74,188700.0,24030.0,43929.0,-0.14,14.59,23.28,41.28,89.0,0.55,1.37,23.74,X40N,ATH,AC
84,VOLTAS,1918.49,-0.24,55.0,H-MC,3.16,209160.0,17418.0,78623.0,-0.04,9.08,37.59,50.08,99.0,0.22,1.52,16.50,XY25,ATH,AC
24,DABUR,735.00,-2.58,60.0,H-MC,6.00,212160.0,15132.0,74489.0,-0.64,7.68,35.11,45.49,102.0,0.20,1.55,21.72,XY24,BTT,FMCG
18,CAMS,5211.76,-5.38,50.0,H-SC,4.82,107584.0,5580.0,38343.0,-1.25,5.47,35.64,43.06,122.0,0.15,0.78,23.52,X40N,ATH,MISC


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,247.44,-42.83,71.0,H-MC,3.48,113658.0,13154.0,17481.0,2.54,13.09,15.38,30.49,79.0,0.75,0.83,49.73,MH,ATH,METALS
36,INDIAMART,4850.92,-50.35,55.0,H-SC,12.18,138489.0,15153.0,118616.0,1.42,12.29,85.65,108.45,119.0,0.13,1.01,37.39,AR,ATH,MISC
85,WHIRLPOOL,2270.00,-41.04,51.0,M-SC,1.98,99742.0,8244.0,70508.0,-2.91,9.01,70.69,86.07,223.0,0.12,0.73,44.84,XR,NTT,DURABLES
86,WIPRO,420.00,-10.62,56.0,M-LC,6.26,157207.0,6262.0,103191.0,-0.86,4.15,65.64,72.51,53.0,0.06,1.14,10.42,XR,NTT,IT
12,BANKINDIA,190.00,-29.19,64.0,H-MC,5.51,181838.0,2030.0,110376.0,0.63,1.13,60.70,62.52,88.0,0.02,1.32,35.60,XR,NTT,BANKS


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ALKYLAMINE,4546.37,-5.01,44.0,H-SC,14.32,88348.0,-12615.0,107148.0,-0.69,-12.49,121.28,93.63,148.0,-0.12,0.64,31.59,SR,ATH,CHEMICALS
22,COFFEEDAY,80.00,-41.65,56.0,L-SC,43.35,87840.0,-25709.0,65757.0,-1.93,-22.64,74.86,35.27,268.0,-0.39,0.64,114.99,XR,NTT,HOTELS
48,MASFIN,398.61,-16.44,56.0,H-SC,13.06,94740.0,-3240.0,24841.0,0.45,-3.31,26.22,22.05,152.0,-0.13,0.69,37.71,XR,ATH,FINANCE
49,NATIONALUM,247.44,-42.83,71.0,H-MC,3.48,113658.0,13154.0,17481.0,2.54,13.09,15.38,30.49,79.0,0.75,0.83,49.73,MH,ATH,METALS
82,VALIANTORG,838.00,-325.20,54.0,H-SC,8.41,125273.0,-38332.0,159648.0,-2.36,-23.43,127.44,74.15,139.0,-0.24,0.91,60.58,XR,NTT,CHEMICALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.0,-41.65,56.0,L-SC,43.35,87840.0,-25709.0,65757.0,-1.93,-22.64,74.86,35.27,268.0,-0.39,0.64,114.99,XR,NTT,HOTELS
60,SAMMAANCAP,326.0,-171.59,64.0,M-SC,3.02,82092.0,-20118.0,113509.0,0.06,-19.68,138.27,91.37,208.0,-0.18,0.60,33.26,XY25,NTT,FINANCE
27,GREENPANEL,537.0,250.78,78.0,M-SC,6.14,156575.0,-26503.0,105485.0,17.61,-14.48,67.37,43.14,230.0,-0.25,1.14,47.28,XY24,NTT,MISC
75,TITAGARH,1548.0,-4.07,61.0,H-SC,6.52,157395.0,-34620.0,113498.0,-1.04,-18.03,72.11,41.08,158.0,-0.31,1.15,33.37,XY24,NTT,ENGINEERING
53,RAJESHEXPO,518.0,1873.48,68.0,L-SC,6.10,53859.0,-83318.0,83411.0,1.62,-60.74,154.87,0.07,267.0,-1.00,0.39,33.95,OX40N,NTT,JEWELLERY


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.71
1,25,43.48
2,50,73.85


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.51
LC    34.43
MC    23.03
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.94
X40      15.26
X40N     12.96
XR       10.06
XY25      9.77
AR        8.62
OX40N     8.24
X200      1.73
MH        1.71
X5K       1.41
SR        1.27
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    27.38
H-LC    26.26
H-MC    20.09
M-SC    13.56
M-LC     7.11
M-MC     2.63
L-SC     1.57
L-LC     1.06
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.39,-14.37,79.39
FMCG,11.91,-3.49,39.70
FINANCE,9.60,-16.74,62.46
BANKS,7.83,-11.40,62.68
MISC,6.10,-13.17,77.78
PAINTS,5.91,-13.81,39.28
ELECTRICAL,5.57,-7.71,67.07
AC,3.60,2.24,30.87
INSURANCE,3.39,-7.98,45.02


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2871420.0,22
AR,1312151.0,9
XR,1230229.0,13
X40,872469.0,10
X40N,631388.0,10
OX40N,515734.0,10
XY25,409916.0,7
SR,246725.0,2
X5K,123673.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3353100.0,28
H-LC,1387564.0,15
M-SC,1385629.0,17
H-MC,1194571.0,15
M-LC,401115.0,5
M-MC,315991.0,2
L-SC,245152.0,3
L-MC,59953.0,1
L-LC,38771.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1041668.0      6
M-SC       XY24       799029.0      7
H-SC       AR         780880.0      5
           XR         677509.0      6
H-LC       X40        581951.0      5
H-MC       XY24       485287.0      4
H-LC       AR         369840.0      2
M-MC       XY24       315991.0      2
H-SC       X40N       298818.0      4
           SR         246725.0      2
           OX40N      244258.0      4
H-LC       X40N       222900.0      4
H-MC       X40        219704.0      4
           XY25       187375.0      2
M-LC       XY24       163734.0      2
L-SC       XR         161741.0      2
M-SC       AR         161431.0      2
M-LC       X5K        123673.0      1
M-SC       OX40N      123387.0      4
           XR         117459.0      2
           XY25       113509.0      1
H-MC       XR         110376.0      1
           X40N       109670.0      2
M-LC       XR         103191.0      1
H-LC       X200        87418.0      1
L-SC       OX40N       83411.0      1
M-SC       X40         70814.0      1
H-LC       XY24        65711.0      1
H-MC       OX40N       64678.0      1
H-SC       MH          63242.0      1
L-MC       XR          59953.0      1
H-LC       XY25        59744.0      2
L-LC       XY25        38771.0      1
H-MC       MH          17481.0      1
M-LC       XY25        10517.0      1

In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
